In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import os

from langchain_ollama import OllamaLLM

In [10]:
ollama_url = os.getenv('OLLAMA_URL')
ollama_url

'http://ollama:11434'

In [11]:
model = 'llama3.2'

In [14]:
llm = OllamaLLM(model=model, base_url=ollama_url)

In [15]:
llm.invoke('Hi')

'How can I assist you today?'